# Excercise - Creating our own custom Model

This is a notebook that provides a quick overview of how to create your own custom model. You will be creating a simple model.
You will be utilizing Keras and Tensorflow


## Water Quality Dataset

This dataset contains water quality measurements and assessments related to potability, which is the suitability of water for human consumption. The dataset's primary objective is to provide insights into water quality parameters and assist in determining whether the water is potable or not. Each row in the dataset represents a water sample with specific attributes, and the "Potability" column indicates whether the water is suitable for consumption.

https://www.kaggle.com/datasets/uom190346a/water-quality-and-potability?select=water_potability.csv


Display the data's first 5 rows

In [1]:

from matplotlib import pyplot
import pandas as pd
import numpy as np

path = './data/water_potability.csv'

df = pd.read_csv(path)

df.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0


Split the data into features and targets

In [2]:
# Handle missing values (for simplicity, we'll fill them with the mean of the column)
df = df.replace('?', np.nan).astype(float)
df = df.fillna(df.mean())


# Split the data into features and target
# make them numpy arrays
X = df.drop('Potability', axis=1).values
y = df['Potability'].values

Split the data into train and test sets

In [3]:

# split into train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# further split the train set into train and validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 0)


print("Train shapes: ")
print(X_train.shape)
print(y_train.shape)

print("Test shapes: ")
print(X_test.shape)
print(y_test.shape)

print("Validation shapes: ")
print(X_val.shape)
print(y_val.shape)


Train shapes: 
(2096, 9)
(2096,)
Test shapes: 
(656, 9)
(656,)
Validation shapes: 
(524, 9)
(524,)


# Define the model

In [4]:
# import the necessary libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Define the model
model = Sequential()

# Add the first hidden layer
model.add(Dense(
    units = 128,
    activation = 'relu',
    input_dim = X_train.shape[1]
))

# Add the second hidden layer
model.add(Dense(
    units = 64,
    activation = 'relu',
))

# Add the output layer with 2 neurons
model.add(Dense(
    units = 1,
    activation = 'sigmoid'
))

# Compile the model
optimzer = Adam(learning_rate = 0.001)
model.compile(
    optimizer = optimzer,
    loss = 'mean_squared_error',
    metrics = ['mean_absolute_error']
)

/Users/macbook2015/ml/water_quality_model/.venv/lib/python3.8/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
2024-06-04 07:12:04.246831: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Start the training Process

In [5]:
# Normalize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train the model
history = model.fit(
    X_train,
    y_train,
    validation_data = (X_val, y_val),
    epochs = 200,
    batch_size = 32,
    verbose = 1
)

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print("Test Loss: ", loss)
print("Test Accuracy: ", mae)

Epoch 1/200
66/66 [==============================] - 1s 7ms/step - loss: 0.2382 - mean_absolute_error: 0.4768 - val_loss: 0.4027 - val_mean_absolute_error: 0.4027
Epoch 2/200
66/66 [==============================] - 0s 3ms/step - loss: 0.2213 - mean_absolute_error: 0.4569 - val_loss: 0.5483 - val_mean_absolute_error: 0.5519
Epoch 3/200
66/66 [==============================] - 0s 3ms/step - loss: 0.2095 - mean_absolute_error: 0.4334 - val_loss: 0.5973 - val_mean_absolute_error: 0.5973
Epoch 4/200
66/66 [==============================] - 0s 3ms/step - loss: 0.2016 - mean_absolute_error: 0.4197 - val_loss: 0.5954 - val_mean_absolute_error: 0.5954
Epoch 5/200
66/66 [==============================] - 0s 3ms/step - loss: 0.1972 - mean_absolute_error: 0.4070 - val_loss: 0.5935 - val_mean_absolute_error: 0.5936
Epoch 6/200
66/66 [==============================] - 0s 3ms/step - loss: 0.1954 - mean_absolute_error: 0.4018 - val_loss: 0.5992 - val_mean_absolute_error: 0.5992
Epoch 7/200
66/66 [===

Compare the output without Early stopping